In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Einlesen der Rohdaten

In [4]:
questions = pd.read_excel("./ressources/Umfragedaten.xlsx", sheet_name="Fragen")
data = pd.read_excel("./ressources/Umfragedaten.xlsx", sheet_name="Antworten")


In [5]:
questions.head()

,Frage,Gruppe,Text,1,2,3,4,5,Levels
0,A1.1,Qualität der Vorlesung allgemein,Inhalt,zu schwer,NaN,NaN,NaN,zu einfach,5.0
1,A1.2,Qualität der Vorlesung allgemein,Inhalt,verständlich,NaN,NaN,NaN,abstrakt,5.0
2,A1.3,Qualität der Vorlesung allgemein,Industriebezug,nah,NaN,NaN,NaN,fern,5.0
3,A1.4,Qualität der Vorlesung allgemein,Inhalte aktuell,stimme zu,NaN,NaN,NaN,lehne ab,5.0
4,A2,Qualität der Vorlesung allgemein,Es werden Verbindungen zu anderen Gebieten auf...,sehr oft,NaN,NaN,NaN,nie,5.0


In [6]:
questions = questions.set_index("Frage")
questions[[2, 3, 4]] = questions[[2, 3, 4]].fillna("")
questions["Levels"] = questions["Levels"].astype("Int32") # ganzzahlige Werte mit NaNs
questions.head()

,Gruppe,Text,1,2,3,4,5,Levels
Frage,,,,,,,,
A1.1,Qualität der Vorlesung allgemein,Inhalt,zu schwer,,,,zu einfach,5
A1.2,Qualität der Vorlesung allgemein,Inhalt,verständlich,,,,abstrakt,5
A1.3,Qualität der Vorlesung allgemein,Industriebezug,nah,,,,fern,5
A1.4,Qualität der Vorlesung allgemein,Inhalte aktuell,stimme zu,,,,lehne ab,5
A2,Qualität der Vorlesung allgemein,Es werden Verbindungen zu anderen Gebieten auf...,sehr oft,,,,nie,5


In [7]:
data.head(2)

,A1.1,A1.2,A1.3,A1.4,A2,A3,A Bemerkungen,B5,B6,B7a1,...,D Bemerkungen,E13.1,E13.2,E13.3,F14.1,F14.2,F14.3,F15,F16,F17
0,2,3,2.0,2,1,4,NaN,2,3,1,...,NaN,4,3,2,2,1.0,1,Visualisierung von Daten mit Diagrammen,NaN,NaN
1,3,2,3.0,1,2,3,Die Mathematik verstehe ich nur so halb,2,1,1,...,NaN,5,2,1,1,1.0,1,"Themenauswahl, Didaktik, Übungen mit Leitfaden",Tempo,Durchatmen :-)


Die Ankreuzfragen sind anders zu handhaben als die Freitextaufgaben.
Daher trennen wir den Datensatz in die numerischen und die nicht-numerischen Spalten.
Zusätzlich wird der Datensatz ins Langformat gebracht.

In [8]:
data_num  = data.select_dtypes(include=np.number).melt(var_name="Frage")
data_text = data.select_dtypes(exclude=np.number).melt(var_name="Frage")

Manche numerischen Spalten sind vom Typ "int", andere vom Typ "float".
Eigentlich sind alle ganzzahlig, aber nur float kann NaNs handhaben.
Wir casten zu Integer mit NA-Option.

In [9]:
data_num["value"] = data_num["value"].astype("Int64")

In [10]:
sns.set_theme()
# [data_num["Frage"] == "A1.1"]
for (group_name, sub_df) in data_num.groupby("Frage"):
    mean_value = sub_df["value"].mean()
    num_levels = questions.loc[group_name, "Levels"]
    categories = range(1, num_levels+1)
    sub_df["value"] = sub_df["value"].astype(
        "category").cat.set_categories(categories, ordered=True)

    g = sns.catplot(data=sub_df, x="value", kind="count",
                    color="skyblue", edgecolor="black", height=4, aspect=1.8)
    g.refline(x=mean_value - 1, color="red")
    #g.ax.vlines(mean_value-1, ymin=0, ymax=1, transform=g.ax.get_xaxis_transform(), color = "red")
    g.set_xticklabels(questions.loc[group_name, categories])
    g.set(title=questions.loc[group_name, "Gruppe"] + "\n" + group_name + ": " + questions.loc[group_name, "Text"], xlabel="", ylabel="",
          ylim=(0, 25))
    # show plot
    # plt.show()
    # save plot to temp folder
    g.savefig("./temp/"+group_name + ".png")
    plt.close()


In [11]:
data_text = data_text.query("value.notna()").reset_index(drop="True")

## Frage F14.3

In [17]:
grades = data_text.query("Frage == 'F14.3'").copy()
# grades.value_counts("value", sort=False)
grades["value"] = grades["value"].replace(
    {"1-": "1.3", "2+": "1.7", "3+": "2.7"}).astype(float)

grades.value_counts("value", sort=False)


value
1.0    7
1.3    4
1.4    1
1.7    6
2.0    6
2.7    1
dtype: int64

In [13]:
g = sns.displot(x=grades["value"], kind="hist", color="skyblue", edgecolor="black",
                binwidth=1/3, binrange=(1-1/6, 4+1/6), height=4, aspect=1.8)
g.ax.vlines(grades["value"].mean(), ymin=0, ymax=1,
            transform=g.ax.get_xaxis_transform(), color="red")
g.set(title=questions.loc["F14.3", "Gruppe"] + "\n" + "F14.3" + ": " +
      questions.loc["F14.3", "Text"], xlabel="", ylabel="", ylim=(0, 25))
g.savefig("./temp/_Final_F14.3.png")
plt.close()


## Freitextkommentare

In [14]:
comments = data_text.query("Frage != 'F14.3'")
comments.head()


,Frage,value
0,A Bemerkungen,Die Mathematik verstehe ich nur so halb
1,A Bemerkungen,Angenehmes Tempo mit passender Menge an Inform...
2,A Bemerkungen,Daumen hoch (gemalt)
3,A Bemerkungen,Ich würde gerne etwas zu Reinforcement Learnin...
4,A Bemerkungen,Eventuell etwas tiefer in die einzelnen Pakete...


In [15]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)

(comments.merge(questions[["Gruppe", "Text"]], left_on="Frage", right_on="Frage")[["Frage", "Gruppe", "Text", "value"]]
    .style.background_gradient(low=1, high = 5, axis = "index"))

,Frage,Gruppe,Text,value
0,A Bemerkungen,Qualität der Vorlesung allgemein,nan,Die Mathematik verstehe ich nur so halb
1,A Bemerkungen,Qualität der Vorlesung allgemein,nan,Angenehmes Tempo mit passender Menge an Information
2,A Bemerkungen,Qualität der Vorlesung allgemein,nan,Daumen hoch (gemalt)
3,A Bemerkungen,Qualität der Vorlesung allgemein,nan,Ich würde gerne etwas zu Reinforcement Learning lernen
4,A Bemerkungen,Qualität der Vorlesung allgemein,nan,Eventuell etwas tiefer in die einzelnen Pakete gehen und mehr Übungsinhalte dazu
5,A Bemerkungen,Qualität der Vorlesung allgemein,nan,etwas zu schnell
6,A Bemerkungen,Qualität der Vorlesung allgemein,nan,Einblicke in R & Fortran währen cool
7,A Bemerkungen,Qualität der Vorlesung allgemein,nan,Geschwindigkeit ist ab und zu sehr hoch
8,A Bemerkungen,Qualität der Vorlesung allgemein,nan,"Es war so, wie ich mir den Kurs vorgestellt habe."
9,B Bemerkungen,Didaktische Aufbereitung,nan,"Finde es gut, dass wir in den Übungen zuerst selbst arbeiten müssen und die Aufgabe danach Schritt für Schritt erklärt wird. Das gemeinsame Erarbeiten/Mitschreiben auf der Tafel ist eine gute Lösung. Teilweise nur etwas schnell."
